<a href="https://colab.research.google.com/github/3000minsu/2023-LG-Aimers/blob/main/4_XGboost(classifier_%EC%88%98%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [3]:
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

Mounted at /content/drive


In [4]:
train = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']
train_y_quality = train_df['Y_Quality']
test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [5]:
over = train_df[train_df['Y_Class'] == 2]['Y_Quality']
normal = train_df[train_df['Y_Class'] == 1]['Y_Quality']
under = train_df[train_df['Y_Class'] == 0]['Y_Quality']


In [6]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')
train


Done.


,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#그래서 하나의 세트에서 모두 결측치인 열은 두 세트에서 모두 삭제 
#data leakage 인가?

invalid=[]
for col in train.columns:
  if(train[col].isnull().all()):
    invalid+=[col]

train = train.drop(columns=invalid)
test = test.drop(columns=invalid)

In [8]:
RF =  XGBRegressor(objective='reg:squarederror', eta=0.005)
RF.fit(train, train_y_quality)

print('Done.')

Done.


In [9]:
real_preds =[]
preds = RF.predict(test)
for i in range(len(preds)):
  if normal.min() < preds[i] < normal.max():
        preds[i]=1
  elif preds[i] <= under.max():
        preds[i]=0
  elif over.min() <= preds[i]:
       preds[i]=2
  else:
      print(preds[i])
      preds[i]=0
     
print('Done.')

0.5349128
0.5349447
Done.


In [10]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [11]:
submit['Y_Class'] = preds

In [12]:
submit.to_csv('./baseline_submission.csv', index=False)